# Training a ride duration prediction model

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error
import mlflow
from mlflow.models import infer_signature
import xgboost as xgb
from hyperopt import hp, tpe, Trials, STATUS_OK, fmin
from hyperopt.pyll import scope

In [2]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/08/24 23:35:36 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/08/24 23:35:36 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/08/24 23:35:36 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1756071336858, experiment_id='1', last_update_time=1756071336858, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
def one_hot_encoding(df, categories, enc=None):
	df_categorical = df[categories].reset_index(drop=True)
	X_numerical = df.drop(categories, axis=1).reset_index(drop=True)

	if not enc:
		enc = OneHotEncoder(dtype=np.int32, handle_unknown='ignore')
		enc.fit(df_categorical)

	X_categorical = pd.DataFrame(
		data=enc.transform(df_categorical).toarray(),
		columns=enc.get_feature_names_out()
	)

	X = pd.concat([X_numerical, X_categorical], axis=1)

	return X, enc

In [4]:
def preprocessing(url, categorical, numerical, enc=None):
	# read in the raw data
	df = pd.read_parquet(url)
	df[categorical] = df[categorical].astype(str)

    # compute duration of a trip in minutes and minutes
	df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime) / pd.Timedelta(minutes=1)
	df = df.loc[(df.duration >= 1) & (df.duration <= 60)]

	# df['PU_DO'] = df.PULocationID.str.cat(df.DOLocationID, sep='_')
	# categorical = ['PU_DO']

	df = df[categorical + numerical + ['duration']]
	X, enc = one_hot_encoding(df, categorical, enc=enc)

	y = X.duration
	X.drop('duration', axis=1, inplace=True)

	return X.values, y.values, enc

In [5]:
url_train = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet"
url_val = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet"

In [6]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [7]:
X_train, y_train, enc = preprocessing(
	url=url_train, categorical=categorical, numerical=numerical, enc=None)
X_val, y_val, _ = preprocessing(
	url=url_val, categorical=categorical, numerical=numerical, enc=enc)

### Logging Basics

In [8]:
mlflow.xgboost.autolog(disable=True)

In [9]:
alpha = .01
lasso = Lasso(alpha)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_val)
rmse = root_mean_squared_error(y_val, y_pred)

In [10]:
with mlflow.start_run():
	mlflow.set_tag("developer", "bastien winant")

	mlflow.log_param("train-data", url_train)
	mlflow.log_param("valid-data", url_val)

	mlflow.log_param("alpha", alpha)

	mlflow.log_metric("rmse", rmse)

### Hyperparameter Tuning

In [11]:
train = xgb.DMatrix(data=X_train, label=y_train)
valid = xgb.DMatrix(data=X_val, label=y_val)

In [12]:
def objective(params):
	booster = xgb.train(
		params=params,
		dtrain=train,
		num_boost_round=1000,
		evals=[(valid, "validation")],
		early_stopping_rounds=50
	)

	y_pred = booster.predict(valid)
	rmse = root_mean_squared_error(y_val, y_pred)

	with mlflow.start_run():
		mlflow.set_tag("model", "xgboost")
		mlflow.log_params(params)
		mlflow.log_metric("rmse", rmse)

	return {'loss': rmse, 'status': STATUS_OK}

In [13]:
search_space = {
	'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
	'learning_rate': hp.loguniform('learning_rate', -3, 0),
	'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
	'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
	'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
	'objective': 'reg:squarederror',
	'seed': 42
}

In [14]:
best_result = fmin(
	fn=objective,
	space=search_space,
	algo=tpe.suggest,
	max_evals=50,
	trials=Trials()
)

[0]	validation-rmse:8.15301                           
[52]	validation-rmse:6.80172                          
[0]	validation-rmse:10.72553                                                   
[69]	validation-rmse:6.61237                                                   
[0]	validation-rmse:7.40636                                                    
[93]	validation-rmse:6.29664                                                   
[0]	validation-rmse:11.60843                                                   
[105]	validation-rmse:6.56766                                                  
[0]	validation-rmse:11.53458                                                   
[101]	validation-rmse:6.47210                                                  
[0]	validation-rmse:11.27204                                                   
[103]	validation-rmse:6.38019                                                  
[0]	validation-rmse:7.28120                                                    
[63]	valid

### Autologging

In [16]:
mlflow.xgboost.autolog(log_input_examples=True, log_model_signatures=False)

In [17]:
# validation error-minimizing hyperparameters
best_params = {
	'max_depth': 4,
	'learning_rate': 0.37954383651795515,
	'reg_alpha': 0.03213702122011809,
	'reg_lambda': 0.00932527914188104,
	'min_child_weight': 4.774415054918944,
	'seed': 42
}

In [18]:
booster = xgb.train(
	params=best_params,
	dtrain=train,
	num_boost_round=1000,
	evals=[(valid, "validation")],
	early_stopping_rounds=50
)

2025/08/24 23:51:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '676c162f4e49437ca9cf21eb432b1491', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:9.45528
[1]	validation-rmse:8.06779
[2]	validation-rmse:7.40706
[3]	validation-rmse:7.08455
[4]	validation-rmse:6.93237
[5]	validation-rmse:6.84670
[6]	validation-rmse:6.80047
[7]	validation-rmse:6.76758
[8]	validation-rmse:6.74005
[9]	validation-rmse:6.72419
[10]	validation-rmse:6.70471
[11]	validation-rmse:6.69264
[12]	validation-rmse:6.67897
[13]	validation-rmse:6.66988
[14]	validation-rmse:6.65602
[15]	validation-rmse:6.64536
[16]	validation-rmse:6.63240
[17]	validation-rmse:6.62283
[18]	validation-rmse:6.61542
[19]	validation-rmse:6.61070
[20]	validation-rmse:6.60445
[21]	validation-rmse:6.59357
[22]	validation-rmse:6.58976
[23]	validation-rmse:6.58236
[24]	validation-rmse:6.57980
[25]	validation-rmse:6.57198
[26]	validation-rmse:6.56780
[27]	validation-rmse:6.56307
[28]	validation-rmse:6.55902
[29]	validation-rmse:6.55568
[30]	validation-rmse:6.55274
[31]	validation-rmse:6.54832
[32]	validation-rmse:6.54106
[33]	validation-rmse:6.53594
[34]	validation-rmse:6.5

2025/08/24 23:51:41 WARNING mlflow.xgboost: Failed to gather input example: please ensure that autologging is enabled before constructing the dataset.
2025/08/24 23:51:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/24 23:51:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:51:42] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/08/24 23:51:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
